In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import opttrot


## Fast Pauli decomposition algorithm implementation

```
Hantzko, Binkowski, and Gupta, Tensorized Pauli decomposition algorithm
```

In [3]:
import numpy as np
from copy import deepcopy

In [4]:
from opttrot import utils 

In [5]:
# Size = 2^n Hermit matrix
n =5
size_n = int(2**n)
A = np.asmatrix(np.random.rand(size_n, size_n).astype(complex))
ImA = np.asmatrix(np.random.rand(size_n, size_n).astype(complex))
A = A.H@A
ImA = ImA.H@ImA
H = A+1j*ImA

In [6]:
pauli_matrix, pauli_terms = opttrot.Hamiltonian.generate_pauli_terms(n) # Get 2^n pauli-terms

In [7]:
# Frobenius inner product
decomposition = {}
for matrix, term in zip(pauli_matrix, pauli_terms):
    coef = utils.frobenius_inner(matrix, H)
    decomposition[term] = coef

In [8]:
decomposition

{'IIIII': (10.945459884997984+10.590954664796003j),
 'IIIIZ': (-0.22156279836210535-0.4747238410695145j),
 'IIIZI': (0.025874282712689656-0.055183512478206964j),
 'IIIZZ': (-0.32668552406580753-0.053061805184501765j),
 'IIZII': (-0.2452242275644353-0.2670992804898942j),
 'IIZIZ': (-0.15356517711237005+0.17201095673579914j),
 'IIZZI': (-0.020096071514722924-0.12592299399276163j),
 'IIZZZ': (-0.13367736513616746+0.3699756232067051j),
 'IZIII': (-0.1503411636847784-0.1091367496206353j),
 'IZIIZ': (-0.2052215360652207+0.6837500355949917j),
 'IZIZI': (0.06917665508645793+0.043182903089829805j),
 'IZIZZ': (-0.37376877938013137-0.3321075509576826j),
 'IZZII': (-0.476828193981215+0.20420643369691133j),
 'IZZIZ': (-0.1243502778032548-0.16815981706650668j),
 'IZZZI': (-0.08796981204727228+0.09758720149417405j),
 'IZZZZ': (0.02321880933650744+0.22680390633959674j),
 'ZIIII': (-0.050114299965577425-0.2225694975208019j),
 'ZIIIZ': (-0.3280815007769781-0.28081263118200217j),
 'ZIIZI': (-0.1266778178

In [9]:
H_test = deepcopy(H)

### Implementation

In [10]:
H_test = deepcopy(H)
exp2n = H_test.shape[0]
n = int(np.log2(exp2n))

print("Hamiltonian shape:", H_test.shape)
print("Total qubits:", n)


Hamiltonian shape: (32, 32)
Total qubits: 5


In [11]:
# Implementation of decomposition
l = int(exp2n)
for i in range(n): #range(n)
    m = int(2**i)
    l = int(l/2)
    #print("number of submatrix:", m)
    #print("sub matrix unit:", l)
    for j in range(m):
        for k in range(m):

            num_i = j*(2*l)
            num_j = k*(2*l)

            #print(f"{num_i}, {num_j}")
            #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")
            
            H_test[num_i: num_i+l, num_j:num_j+l] += H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
            H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H_test[num_i: num_i+l, num_j:num_j+l] - 2*H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

            #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
            H_test[num_i: num_i+l, num_j+l:num_j+2*l] += H_test[num_i+l: num_i+2*l, num_j:num_j+l] 
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] =  H_test[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H_test[num_i+l: num_i+2*l, num_j:num_j+l]
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
    #print("------------------------------------------------")
    H_test *= 0.5


In [12]:
def tensorized_decomposition(H):
    exp2n = H.shape[0]
    n = int(np.log2(exp2n)) # wires

    l = int(exp2n)

    for i in range(n): #range(n)
        m = int(2**i) # Number of submatrix
        l = int(l/2) # Sub matrix size, square
        for j in range(m):
            for k in range(m):
                num_i = j*(2*l) # Initial position of sub matrix row
                num_j = k*(2*l) # Initial position of sub matrix column
                #print(f"{num_i}, {num_j}")
                #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")

                H[num_i: num_i+l, num_j:num_j+l]        += H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
                H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] = H[num_i: num_i+l, num_j:num_j+l] - 2*H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

                #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
                H[num_i: num_i+l, num_j+l:num_j+2*l] += H[num_i+l: num_i+2*l, num_j:num_j+l] 
                H[num_i+l: num_i+2*l, num_j:num_j+l] =  H[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H[num_i+l: num_i+2*l, num_j:num_j+l]
                H[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
        H *= 0.5
    return H

In [13]:
def index_xzcode(i, j):
    return i^j, i
def xzcode_index(nx, nz):
    return nz, nx^nz

In [14]:
opttrot.utils.pstr_from_xz(2, 4)

'ZXI'

In [15]:
H_test = deepcopy(H)

In [16]:
H_decom = tensorized_decomposition(H_test)
nr, nc = H_decom.shape
decom = {}
for i in range(nr):
    for j in range(nc):
        coef = H_decom[i, j]
        decom[opttrot.utils.xz_fam_code_to_pstr(index_xzcode(i,j), n)] = coef
        

In [17]:
nr, nc = H_test.shape
decom = {}
for i in range(nr):
    for j in range(nc):
        coef = H_test[i, j]
        decom[opttrot.utils.xz_fam_code_to_pstr(index_xzcode(i,j), n)] = coef
        

In [18]:
decom

{'IIIII': (10.945459884997984+10.590954664796001j),
 'IIIIX': (8.434273895325958+8.056765853167535j),
 'IIIXI': (8.35105044913163+7.8499428988124995j),
 'IIIXX': (8.254779098631358+7.605544657436686j),
 'IIXII': (8.396196628460476+8.059023677539532j),
 'IIXIX': (8.119185241408417+7.855051391450978j),
 'IIXXI': (8.258394405672714+7.99329646212456j),
 'IIXXX': (8.104867053620927+7.776637233245777j),
 'IXIII': (8.421869894294773+7.851892668608022j),
 'IXIIX': (8.371456807814887+7.780231608457198j),
 'IXIXI': (8.156757622154561+7.900908892744351j),
 'IXIXX': (8.449141116317964+7.7344478553693765j),
 'IXXII': (8.341611755136443+7.73385233638245j),
 'IXXIX': (8.227866391189162+7.863149144436983j),
 'IXXXI': (8.266358780581402+7.874813094447777j),
 'IXXXX': (8.243862687883855+7.888086891778787j),
 'XIIII': (8.29634307277971+7.978048807496786j),
 'XIIIX': (8.499147840025216+7.854942701071317j),
 'XIIXI': (8.185819811740664+7.994381071366483j),
 'XIIXX': (8.32253067438138+7.719756165109903j),
 

In [19]:
diff = {}
for key in decom.keys():
    diff[key] = abs(decomposition[key] - decom[key])

In [20]:
np.array(list(diff.values())).max()

2.5121479338940403e-15

In [21]:
from pd import PauliDecomposition

In [22]:
pstring, pcoef = PauliDecomposition(H)
pdecompose = {st:coef for st, coef in zip(pstring, pcoef)}

In [23]:
pdecompose

{'IIIII': (10.945459884997984+10.590954664796001j),
 'IIIIX': (8.434273895325958+8.056765853167535j),
 'IIIIZ': (-0.2215627983621049-0.4747238410695145j),
 'IIIXI': (8.35105044913163+7.8499428988124995j),
 'IIIXX': (8.254779098631358+7.605544657436686j),
 'IIIXZ': (-0.16408133564076977-0.5423567724018161j),
 'IIIYY': (0.4083272122799557+0.23590864524452915j),
 'IIIZI': (0.025874282712688768-0.05518351247820563j),
 'IIIZX': (0.048620706576230965-0.4118085315534099j),
 'IIIZZ': (-0.3266855240658071-0.05306180518450132j),
 'IIXII': (8.396196628460476+8.059023677539532j),
 'IIXIX': (8.119185241408417+7.855051391450978j),
 'IIXIZ': (-0.046746661151936486-0.5798830686452945j),
 'IIXXI': (8.258394405672714+7.99329646212456j),
 'IIXXX': (8.104867053620927+7.776637233245777j),
 'IIXXZ': (-0.07908129813258302-0.5395950650270716j),
 'IIXYY': (0.4955656360724756+0.14650222914228417j),
 'IIXZI': (0.16212332649297645-0.3229173074093654j),
 'IIXZX': (0.3100970688545379-0.2935692957421878j),
 'IIXZZ':

In [24]:
diff2 = {}
for key in decom.keys():
    try:
        diff2[key] = abs(pdecompose[key] - decom[key])
    except:
        print(decom[key])

0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
-4.440892098500626e-16j
0j
(-4.440892098500626e-16+0j)
0j
0j
0j
4.440892098500626e-16j
0j
8.881784197001252e-16j
0j
0j
0j
4.440892098500626e-16j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
(4.440892098500626e-16-2.220446049250313e-16j)
-6.661338147750939e-16j
(-2.220446049250313e-16+0j)
0j
(4.440892098500626e-16-4.440892098500626e-16j)
(-4.440892098500626e-16-4.440892098500626e-16j)
0j
0j
(6.661338147750939e-16-2.220446049250313e-16j)
(4.440892098500626e-16+8.881784197001252e-16j)
(2.220446049250313e-16-2.220446049250313e-16j)
0j
(-4.440892098500626e-16+0j)
(-2.220446049250313e-16+0j)
(-2.220446049250313e-16-4.440892098500626e-16j)
0j
-2.220446049250313e-16j
0j
0j
0j
(4.440892098500626e-16+4.440892098500626e-16j)
0j
0j
0j
(-4.440892098500626e-16+0j)
0j
0j
0j
(6.661338147750939e-16+0j)
0j
0j
0j
(-4.440892098500626e-16+2.220446049250313e-16j)
0j
0j
0j
-4.440892098500626e-16j
0j
0j
0j
(-2.220446049250313e-16+2.220446049250313

In [14]:
def fast_p_decompose(H):
    exp2n = H.shape[0]
    n = int(np.log2(exp2n))
    # Implementation of decomposition
    l = int(exp2n)
    for i in range(n):
        m = int(2**i)
        l = int(l/2)
        for j in range(m):
            for k in range(m):

                num_i = j*l
                num_j = k*l

                H[num_i: num_i+l, num_j:num_j+l] += H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
                H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H[num_i: num_i+l, num_j:num_j+l] - 2*H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

                H[num_i: num_i+l, num_j+l:num_j+2*l] += H[num_i+l: num_i+2*l, num_j:num_j+l] 
                H[num_i+l: num_i+2*l, num_j:num_j+l] =  H[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H[num_i+l: num_i+2*l, num_j:num_j+l]
                H[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j

        H *= 0.5
    return H

# string to matrix

In [103]:
from opttrot.utils import pstr_to_matrix, pstr_to_xz_fam_code, xz_fam_code_to_pstr

In [104]:
pstr= "IZX"
xz_code = pstr_to_xz_fam_code(pstr)
p_matrix = pstr_to_matrix(pstr)

In [105]:
p_matrix

array([[ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -1.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j,
        -0.+0.j]])

In [106]:
xz_code

(1, 2)

In [107]:
mat = np.zeros(p_matrix.shape)
mat[*xzcode_index(*xz_code)] = 1

In [108]:
from copy import deepcopy

In [109]:
mat

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [110]:
tensorized_decomposition(p_matrix)

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [111]:
# 0, 1
# y = (x+1)/2
# y+y-1

In [112]:
# Routines
i,j = p_index = xzcode_index(*xz_code)
print("P index:", p_index)
print("1) I-Z/X-Y:", "X-Y" if (i+j)%2 else "I-Z")
print("2) (1/2, 1/2):", (i%2, j%2))
print("---", "Ori:", (i, j), "Cor:", (i+1-2*(i%2), j+ 1-(2*(j%2))))
pair = ((i, j), (i+1-2*(i%2), j+ 1-(2*(j%2))))

l = 1
print("Size:", l)
print(f"Range:[{l*pair[0][0]}: {l*pair[0][0] +l}, {l*pair[0][1]}:{l*pair[0][1]+l}], [{l*pair[1][0]}: {l*pair[1][0] +l}, {l*pair[1][1]}:{l*pair[1][1]+l}]")
print("New pair:", (i>>1, j>>1)) # Next pair

P index: (2, 3)
1) I-Z/X-Y: X-Y
2) (1/2, 1/2): (0, 1)
--- Ori: (2, 3) Cor: (3, 2)
Size: 1
Range:[2: 3, 3:4], [3: 4, 2:3]
New pair: (1, 1)


In [113]:
mat_copy = deepcopy(mat)

In [114]:
A12 = mat[2:3, 3:4] - 1j*mat[3:4, 2:3]
A21 = mat[2:3, 3:4] + 1j*mat[3:4, 2:3]

### Implementation

From pauli polynomial to Hamiltonian

In [121]:
#P-xz polynomial-------
qubits = 3
pxzcodes = [
    (3,   (1, 4)),
    (9.2, (2, 3)),
    (2.1, (6, 3))
]
#matrix----------------
mat = np.zeros((2**qubits, 2**qubits), dtype=complex)
#P-index--------------
p_indexes = [xzcode_index(*xz_code) for (c, xz_code) in pxzcodes]
for pxz in pxzcodes:
    i, j = xzcode_index(*pxz[1])
    mat[i, j] = pxz[0]
#---------------------

steps = int(np.log2(mat.shape[0]))
unit_size= 1

in_ps = [p for p in p_indexes]
for step in range(steps):
    #print("========================================")
    psteps =[]
    dup = []
    #----
    for p in in_ps:
        i, j = p
        p_class = (i+j)%2 #0: IZ, 1:XY
        n, o = i%2, j%2 # (1), (2) determination
        l, m = (i+1-2*(n), j+ 1-(2*(o))) # get a corresponding location

        if (l,m) in dup: # Eliminate duplicated operation.
            continue
        elif (i, j) in dup:
            dup.append((l,m))
            continue
        else:
            dup.append((i,j))
            dup.append((l,m))
             
        if n: # (2)
            pair = ((l, m), (i, j))
        else: #(1)
            pair = ((i, j), (l, m))
        
        #print((i,j), (l,m))
        #print(dup)
        #print("XY" if p_class else "IZ")
        #print("(2)" if n else "(1)")
        #print("size:", unit_size)
        #print(f"{unit_size*pair[0][0]}:{unit_size*pair[0][0]+unit_size},  {unit_size*pair[0][1]}:{unit_size*pair[0][1]+unit_size}")
        #print(f"{unit_size*pair[1][0]}: {unit_size*pair[1][0]+unit_size}, {unit_size*pair[1][1]}:{unit_size*pair[1][1]+unit_size}")
        
        coef = -1j if p_class else 1 # ture: XY, false: IZ
       
        mat[unit_size*pair[0][0]: unit_size*pair[0][0]+unit_size, unit_size*pair[0][1]:unit_size*pair[0][1]+unit_size] += coef*mat[unit_size*pair[1][0]: unit_size*pair[1][0] +unit_size, unit_size*pair[1][1]:unit_size*pair[1][1]+unit_size]
        mat[unit_size*pair[1][0]: unit_size*pair[1][0]+unit_size, unit_size*pair[1][1]:unit_size*pair[1][1]+unit_size] =       mat[unit_size*pair[0][0]: unit_size*pair[0][0] +unit_size, unit_size*pair[0][1]:unit_size*pair[0][1]+unit_size] -2*coef *mat[unit_size*pair[1][0]: unit_size*pair[1][0] +unit_size, unit_size*pair[1][1]:unit_size*pair[1][1]+unit_size]
        
        i >>=1
        j >>=1
        if (i, j) in psteps:
            continue
        else:
            psteps.append((i,j))
    #----
    in_ps = [p for p in psteps]
    unit_size *=2
    #print(in_ps)
    #print(mat)

[[0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 9.2+0.j 0. +0.j 0. +0.j 0. +0.j 2.1+0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 3. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]]
(4, 5) (5, 4)
[(4, 5), (5, 4)]
XY
(1)
size: 1
4:5,  5:6
5: 6, 4:5
(3, 1) (2, 0)
[(4, 5), (5, 4), (3, 1), (2, 0)]
IZ
(2)
size: 1
2:3,  0:1
3: 4, 1:2
(3, 5) (2, 4)
[(4, 5), (5, 4), (3, 1), (2, 0), (3, 5), (2, 4)]
IZ
(2)
size: 1
2:3,  4:5
3: 4, 5:6
[(2, 2), (1, 0), (1, 2)]
[[ 0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 9.2+0.j  0. +0.j  0. +0.j  0. +0.j  2.1

In [122]:
mat

array([[ 0.+0.j ,  3.+0.j ,  0.-9.2j,  0.+0.j ,  0.+0.j ,  0.+0.j ,
         0.-2.1j,  0.+0.j ],
       [ 3.+0.j ,  0.+0.j ,  0.+0.j ,  0.+9.2j,  0.+0.j ,  0.+0.j ,
         0.+0.j ,  0.+2.1j],
       [ 0.+9.2j,  0.+0.j ,  0.+0.j ,  3.+0.j ,  0.+2.1j,  0.+0.j ,
         0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.-9.2j,  3.+0.j ,  0.+0.j ,  0.+0.j ,  0.-2.1j,
         0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.-2.1j,  0.+0.j ,  0.+0.j , -3.+0.j ,
         0.-9.2j,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+2.1j, -3.+0.j ,  0.+0.j ,
         0.+0.j ,  0.+9.2j],
       [ 0.+2.1j,  0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+9.2j,  0.+0.j ,
         0.+0.j , -3.+0.j ],
       [ 0.+0.j ,  0.-2.1j,  0.+0.j ,  0.+0.j ,  0.+0.j ,  0.-9.2j,
        -3.+0.j ,  0.+0.j ]])

In [123]:
# p_xz codes to matrix
pxzcodes
mat_ref = np.zeros(mat.shape, dtype=complex)
for pxz in pxzcodes:
    coef, xz = pxz
    pstr = xz_fam_code_to_pstr(xz, 3)
    pmat = pstr_to_matrix(pstr)
    mat_ref += coef*pmat
mat_ref

array([[ 0.+0.j ,  3.+0.j ,  0.-9.2j,  0.+0.j ,  0.+0.j ,  0.+0.j ,
         0.-2.1j,  0.+0.j ],
       [ 3.+0.j ,  0.+0.j ,  0.+0.j ,  0.+9.2j,  0.+0.j ,  0.+0.j ,
         0.+0.j ,  0.+2.1j],
       [ 0.+9.2j,  0.+0.j ,  0.+0.j ,  3.+0.j ,  0.+2.1j,  0.+0.j ,
         0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.-9.2j,  3.+0.j ,  0.+0.j ,  0.+0.j ,  0.-2.1j,
         0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.-2.1j,  0.+0.j ,  0.+0.j , -3.+0.j ,
         0.-9.2j,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+2.1j, -3.+0.j ,  0.+0.j ,
         0.+0.j ,  0.+9.2j],
       [ 0.+2.1j,  0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+9.2j,  0.+0.j ,
         0.+0.j , -3.+0.j ],
       [ 0.+0.j ,  0.-2.1j,  0.+0.j ,  0.+0.j ,  0.+0.j ,  0.-9.2j,
        -3.+0.j ,  0.+0.j ]])

In [118]:
# Single Pauli term to matrix

In [ ]:
# Multiple Pauli term at once
# Inverse routine of decomposition

In [ ]:
# Implementation of decomposition
l = int(exp2n)
for i in range(n): #range(n)
    m = int(2**i)
    l = int(l/2)
    #print("number of submatrix:", m)
    #print("sub matrix unit:", l)
    for j in range(m):
        for k in range(m):

            num_i = j*(2*l)
            num_j = k*(2*l)

            #print(f"{num_i}, {num_j}")
            #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")
            
            H_test[num_i: num_i+l, num_j:num_j+l] += H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
            H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H_test[num_i: num_i+l, num_j:num_j+l] - 2*H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

            #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
            H_test[num_i: num_i+l, num_j+l:num_j+2*l] += H_test[num_i+l: num_i+2*l, num_j:num_j+l] 
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] =  H_test[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H_test[num_i+l: num_i+2*l, num_j:num_j+l]
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
    #print("------------------------------------------------")
    H_test *= 0.5
